In [1]:
import pandas as pd

input_path = r"C:\Users\noahi\Downloads\delivery_reliability.csv\delivery_reliability.csv"
output_path = r'C:\Users\noahi\Downloads\scaling_forecasts_STEP1.csv'

# We process 50,000 rows at a time
chunk_size = 50000
is_first_chunk = True

for chunk in pd.read_csv(input_path, chunksize=chunk_size, low_memory=False):
    # 1. Immediate Clean: Remove completely empty rows
    chunk = chunk.dropna(how='all')
    
    # 2. Immediate Clean: Remove duplicates found WITHIN this batch
    chunk = chunk.drop_duplicates()
    
    # 3. Write directly to disk (Append mode 'a')
    chunk.to_csv(output_path, mode='a', index=False, header=is_first_chunk)
    
    is_first_chunk = False

print("✅ Step 1 Complete: Blanks and local duplicates removed without using RAM.")

✅ Step 1 Complete: Blanks and local duplicates removed without using RAM.


In [2]:
import pandas as pd

# Update this path to your new file location
path = r"C:\Users\noahi\Downloads\delivery_reliability.csv\delivery_reliability.csv"

# Read only the header (row 0)
header = pd.read_csv(path, nrows=0)

print("--- Your Dataset Columns ---")
print(header.columns.tolist())

--- Your Dataset Columns ---
['delivery_id', 'timestamp', 'success', 'delay_minutes', 'customer_satisfaction']


In [3]:
# A. Get Medians from our new Step 1 file
math_cols = ['delay_minutes', 'customer_satisfaction']
temp_df = pd.read_csv(output_path, usecols=math_cols)

medians = {}
for col in math_cols:
    # Clean the column to find a real median
    s = pd.to_numeric(temp_df[col], errors='coerce')
    s = s.replace([float('inf'), float('-inf')], pd.NA).dropna()
    medians[col] = s.median()

print(f"📊 Medians found: {medians}")

# B. Apply fix to final file
final_path = r'C:\Users\noahi\Downloads\delivery_data_FINAL.csv'
is_first_chunk = True

for chunk in pd.read_csv(output_path, chunksize=50000):
    # Fix the Math columns
    for col in math_cols:
        chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        # Replace extreme outliers (e.g., > 1 week of delay) with median
        chunk.loc[chunk[col].abs() > 10080, col] = medians[col] 
        chunk[col] = chunk[col].fillna(medians[col])
    
    # Fix the Timestamp
    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], errors='coerce')
    
    chunk.to_csv(final_path, mode='a', index=False, header=is_first_chunk)
    is_first_chunk = False

print("🚀 Step 2 Complete: Final cleaned file created!")

📊 Medians found: {'delay_minutes': np.float64(7.49095903536962), 'customer_satisfaction': np.float64(3.0)}
🚀 Step 2 Complete: Final cleaned file created!


In [4]:
# A. Get Medians from our new Step 1 file
math_cols = ['delay_minutes', 'customer_satisfaction']
temp_df = pd.read_csv(output_path, usecols=math_cols)

medians = {}
for col in math_cols:
    # Clean the column to find a real median
    s = pd.to_numeric(temp_df[col], errors='coerce')
    s = s.replace([float('inf'), float('-inf')], pd.NA).dropna()
    medians[col] = s.median()

print(f"📊 Medians found: {medians}")

# B. Apply fix to final file
final_path = r'C:\Users\noahi\Downloads\delivery_data_FINAL.csv'
is_first_chunk = True

for chunk in pd.read_csv(output_path, chunksize=50000):
    # Fix the Math columns
    for col in math_cols:
        chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        # Replace extreme outliers (e.g., > 1 week of delay) with median
        chunk.loc[chunk[col].abs() > 10080, col] = medians[col] 
        chunk[col] = chunk[col].fillna(medians[col])
    
    # Fix the Timestamp
    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], errors='coerce')
    
    chunk.to_csv(final_path, mode='a', index=False, header=is_first_chunk)
    is_first_chunk = False

print("🚀 Step 2 Complete: Final cleaned file created!")

📊 Medians found: {'delay_minutes': np.float64(7.49095903536962), 'customer_satisfaction': np.float64(3.0)}
🚀 Step 2 Complete: Final cleaned file created!


In [5]:
import pandas as pd

path = r"C:\Users\noahi\Downloads\delivery_data_FINAL.csv" 

# We only load 5 rows to peek at the formatting
preview = pd.read_csv(path, nrows=5)
print("--- Raw Data Preview ---")
print(preview)

# Check if 'delay_minutes' has any impossible values (like we saw before)
# We'll read a slightly larger chunk just for this check
check_math = pd.read_csv(path, usecols=['delay_minutes', 'customer_satisfaction'], nrows=1000)
print("\n--- Quick Math Audit ---")
print(check_math.describe())

--- Raw Data Preview ---
   delivery_id            timestamp  success  delay_minutes  \
0          1.0  2025-03-20 16:26:00      1.0      11.974611   
1          2.0  2022-11-28 02:22:00      1.0       1.193637   
2          3.0  2023-10-13 22:00:00      1.0       9.050722   
3          4.0  2024-07-23 23:55:00      1.0       2.575706   
4          5.0  2025-08-23 12:42:00      1.0       3.078317   

   customer_satisfaction  
0                    5.0  
1                    5.0  
2                    1.0  
3                    5.0  
4                    1.0  

--- Quick Math Audit ---
       delay_minutes  customer_satisfaction
count    1000.000000            1000.000000
mean        7.348064               2.971000
std         4.305070               1.407885
min         0.023085               1.000000
25%         3.554658               2.000000
50%         7.096370               3.000000
75%        10.986690               4.000000
max        14.999648               5.000000


In [6]:
final_path = r"C:\Users\noahi\Downloads\delivery_data_FINAL.csv"

# Load a small sample of the CLEANED data
cleaned_sample = pd.read_csv(final_path, nrows=100)

print("--- Cleaned Data Audit ---")
# This proves the timestamp is now recognized as a datetime object
cleaned_sample['timestamp'] = pd.to_datetime(cleaned_sample['timestamp'])
print(f"Date Range: {cleaned_sample['timestamp'].min()} to {cleaned_sample['timestamp'].max()}")

# This proves the math is safe
print("\n--- Cleaned Math (No more Infinity!) ---")
print(cleaned_sample[['delay_minutes', 'customer_satisfaction']].describe())

--- Cleaned Data Audit ---
Date Range: 2020-01-12 06:21:00 to 2025-09-14 17:58:00

--- Cleaned Math (No more Infinity!) ---
       delay_minutes  customer_satisfaction
count     100.000000             100.000000
mean        7.585615               2.970000
std         4.183811               1.403135
min         0.060910               1.000000
25%         3.943290               2.000000
50%         7.736929               3.000000
75%        10.958187               4.000000
max        14.991491               5.000000


In [7]:
import os

original_size = os.path.getsize(input_path) / (1024 * 1024)
cleaned_size = os.path.getsize(output_path) / (1024 * 1024)

print(f"Original File: {original_size:.2f} MB")
print(f"Cleaned File: {cleaned_size:.2f} MB")
print(f"Rows removed (approx): {len(seen_ids)}")

Original File: 44.01 MB
Cleaned File: 44.73 MB


NameError: name 'seen_ids' is not defined

In [12]:
import pandas as pd

# Let's count the rows in both files
def count_rows(file_path):
    count = 0
    for chunk in pd.read_csv(file_path, chunksize=100000, usecols=[0]):
        count += len(chunk)
    return count

original_rows = count_rows(r"C:\Users\noahi\Downloads\delivery_reliability.csv\delivery_reliability.csv")
cleaned_rows = count_rows(r'C:\Users\noahi\Downloads\delivery_data_FINAL.csv')

print(f"Original Rows: {original_rows:,}")
print(f"Cleaned Rows:  {cleaned_rows:,}")
print(f"Difference:    {original_rows - cleaned_rows} rows removed")

Original Rows: 835,071
Cleaned Rows:  1,670,143
Difference:    -835072 rows removed


In [13]:
import pandas as pd
import os

input_path = r"C:\Users\noahi\Downloads\delivery_reliability.csv\delivery_reliability.csv"
output_path = r'C:\Users\noahi\Downloads\delivery_data_STEP1.csv'

# --- THE FRESH START ---
# This deletes the old "Double" file so we start with a blank slate
if os.path.exists(output_path):
    os.remove(output_path)

seen_ids = set()
is_first_chunk = True

for chunk in pd.read_csv(input_path, chunksize=50000, low_memory=False):
    # 1. Drop rows where the entire row is blank
    chunk = chunk.dropna(how='all')
    
    # 2. Drop duplicates within this batch based on delivery_id
    chunk = chunk.drop_duplicates(subset=['delivery_id'])
    
    # 3. Filter out IDs we've already processed in previous batches
    chunk = chunk[~chunk['delivery_id'].isin(seen_ids)]
    
    # Update our memory of seen IDs
    seen_ids.update(chunk['delivery_id'].tolist())
    
    # 4. Write to disk
    chunk.to_csv(output_path, mode='a', index=False, header=is_first_chunk)
    is_first_chunk = False

print(f"✅ Cleaned! Final unique row count should be around {len(seen_ids):,}")

✅ Cleaned! Final unique row count should be around 833,283


In [14]:
import pandas as pd
import numpy as np

# Path to the file we just deduplicated
input_path = r'C:\Users\noahi\Downloads\delivery_data_STEP1.csv'
final_path = r'C:\Users\noahi\Downloads\delivery_data_FINAL.csv'

# 1. CALCULATE THE MEDIANS (The "Truth")
# We load only the columns we need to save RAM
math_cols = ['delay_minutes', 'customer_satisfaction']
temp_df = pd.read_csv(input_path, usecols=math_cols)

medians = {}
for col in math_cols:
    # Convert to numeric, turn "inf" and text into NaN
    clean_col = pd.to_numeric(temp_df[col], errors='coerce')
    # Replace Infinity with NaN so they don't spoil the median
    clean_col = clean_col.replace([np.inf, -np.inf], np.nan)
    medians[col] = clean_col.median()

print(f"✅ Calculated Medians: {medians}")

# 2. THE FINAL CLEANING PASS
is_first_chunk = True

# We use chunking again just to be safe on your RAM
for chunk in pd.read_csv(input_path, chunksize=100000):
    
    for col in math_cols:
        # Convert the column to numbers
        chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        
        # LOGIC: If a value is ridiculously high (> 10,000) or Infinity, 
        # swap it with the Median we calculated.
        # 10,000 minutes is about 7 days—anything higher is likely an error.
        limit = 10000 if col == 'delay_minutes' else 10 
        
        chunk.loc[chunk[col].abs() > limit, col] = medians[col]
        chunk.loc[np.isinf(chunk[col]), col] = medians[col]
        
        # Fill any remaining blanks (NaNs) with the Median too
        chunk[col] = chunk[col].fillna(medians[col])
        
    # 3. FIX THE TIMESTAMP FORMAT
    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], errors='coerce')
    
    # Save the result
    chunk.to_csv(final_path, mode='a' if not is_first_chunk else 'w', index=False)
    is_first_chunk = False

print(f"🚀 Master Clean Complete! File saved to: {final_path}")

✅ Calculated Medians: {'delay_minutes': np.float64(7.490866960536037), 'customer_satisfaction': np.float64(3.0)}
🚀 Master Clean Complete! File saved to: C:\Users\noahi\Downloads\delivery_data_FINAL.csv


In [15]:
print(pd.read_csv(final_path)[['delay_minutes', 'customer_satisfaction']].describe())

            delay_minutes customer_satisfaction
count              833291                833291
unique             831524                     7
top     7.490866960536037                   3.0
freq                 1760                168435


In [17]:
import pandas as pd

final_path = r'C:\Users\noahi\Downloads\delivery_data_FINAL.csv'

# Load the data one last time
df = pd.read_csv(final_path)

# 1. FORCE Numeric conversion
# This turns "7.49..." into the actual number 7.49
df['delay_minutes'] = pd.to_numeric(df['delay_minutes'], errors='coerce')

# 2. Final Rounding (Optional but makes it cleaner)
# Rounds the messy decimals to 2 places (e.g., 7.49)
df['delay_minutes'] = df['delay_minutes'].round(2)

# 3. Save it back
df.to_csv(final_path, index=False)

# 4. THE REAL TEST
print("--- THE FINAL AUDIT ---")
print(df[['delay_minutes', 'customer_satisfaction']].describe())

--- THE FINAL AUDIT ---
       delay_minutes
count  833283.000000
mean        7.495393
std         4.631865
min      -150.000000
25%         3.750000
50%         7.490000
75%        11.230000
max      1500.000000


In [18]:
import pandas as pd
import numpy as np
import os

input_path = r"C:\Users\noahi\Downloads\scaling_forecasts.csv\scaling_forecasts.csv"
output_path = r'C:\Users\noahi\Downloads\scaling_forecasts_MASTER_CLEAN.csv'

# --- THE FRESH START ---
if os.path.exists(output_path):
    os.remove(output_path)

# 1. CALCULATE THE MEDIANS (One quick pass)
# We calculate medians first so we have a 'sane' number to replace Infinity with
cols_to_fix = ['expansion_cost', 'projected_deliveries', 'required_robots']
temp_df = pd.read_csv(input_path, usecols=cols_to_fix)

medians = {}
for col in cols_to_fix:
    s = pd.to_numeric(temp_df[col], errors='coerce')
    s = s.replace([np.inf, -np.inf], np.nan).dropna()
    medians[col] = s.median()

print(f"📊 Calculated Sanity Medians: {medians}")

# 2. THE STREAMING CLEAN
is_first_chunk = True
for chunk in pd.read_csv(input_path, chunksize=50000):
    
    # Remove blank rows and duplicates
    chunk = chunk.dropna(how='all').drop_duplicates()
    
    for col in cols_to_fix:
        # Force to numeric
        chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        
        # Replace 'inf' or massive placeholders (> 1 billion) with the median
        chunk.loc[chunk[col].abs() > 1e9, col] = medians[col]
        chunk.loc[np.isinf(chunk[col]), col] = medians[col]
        
        # Fill gaps and round to make it professional
        chunk[col] = chunk[col].fillna(medians[col])
        if col == 'required_robots':
            chunk[col] = chunk[col].round(0).astype(int)
            
    # Write to disk
    chunk.to_csv(output_path, mode='a', index=False, header=is_first_chunk)
    is_first_chunk = False

print(f"🚀 Scaling Forecasts are now Clean! Saved to: {output_path}")

📊 Calculated Sanity Medians: {'expansion_cost': np.float64(5499.300698556359), 'projected_deliveries': np.float64(100.0), 'required_robots': np.float64(10.0)}
🚀 Scaling Forecasts are now Clean! Saved to: C:\Users\noahi\Downloads\scaling_forecasts_MASTER_CLEAN.csv


In [19]:
final_df = pd.read_csv(output_path)
print("--- Professional Scaling Audit ---")
print(final_df[['expansion_cost', 'required_robots']].describe())


--- Professional Scaling Audit ---
       expansion_cost  required_robots
count    8.350710e+05     8.350710e+05
mean    -3.354726e+03    -3.763708e+01
std      3.293863e+06     2.969883e+05
min     -9.999997e+08    -1.900000e+08
25%      3.249929e+03     5.000000e+00
50%      5.499301e+03     1.000000e+01
75%      7.746154e+03     1.500000e+01
max      9.999997e+08     1.900000e+08


In [20]:
import pandas as pd
import numpy as np

path = r'C:\Users\noahi\Downloads\scaling_forecasts_MASTER_CLEAN.csv'

# 1. Load the data
df = pd.read_csv(path)

for col in ['expansion_cost', 'required_robots']:
    # Define the "Safe Zone" based on your 25% and 75% marks
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    
    # Anything beyond this is likely a -999,999,999 placeholder
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    # Get the median to use as a replacement
    median_val = df[col].median()
    
    # "Clip" the data: Replace anything outside the bounds with the median
    df.loc[(df[col] < lower_bound) | (df[col] > upper_bound), col] = median_val

# 2. Final Rounding
df['required_robots'] = df['required_robots'].round(0).astype(int)

# 3. Save as the TRUE Professional version
final_path = r'C:\Users\noahi\Downloads\scaling_forecasts_PROFESSIONAL_FINAL.csv'
df.to_csv(final_path, index=False)

print("--- THE FINAL SURGICAL AUDIT ---")
print(df[['expansion_cost', 'required_robots']].describe())

--- THE FINAL SURGICAL AUDIT ---
       expansion_cost  required_robots
count   835071.000000    835071.000000
mean      5498.435262         9.996639
std       2597.313543         5.469420
min       1000.015510         1.000000
25%       3250.057462         5.000000
50%       5499.300699        10.000000
75%       7746.122004        15.000000
max       9999.996599        19.000000


In [23]:
import os

original_path = r"C:\Users\noahi\Downloads\scaling_forecasts.csv\scaling_forecasts.csv"
pro_path = r'C:\Users\noahi\Downloads\scaling_forecasts_PROFESSIONAL_FINAL.csv'

def get_mb(path):
    if os.path.exists(path):
        return f"{os.path.getsize(path) / (1024 * 1024):.2f} MB"
    return "File Not Found"

print(f"{'Dataset Version':<30} | {'File Size':<15}")
print("-" * 50)
print(f"{'Original Raw Data':<30} | {get_mb(original_path):<15}")
print(f"{'Professional Final Data':<30} | {get_mb(pro_path):<15}")


Dataset Version                | File Size      
--------------------------------------------------
Original Raw Data              | 41.61 MB       
Professional Final Data        | 40.75 MB       


In [24]:
import pandas as pd

# Update this path to the new file you want to clean
new_file_path = r"C:\Users\noahi\Downloads\delivery_financials.csv\delivery_financials.csv" 

# Let's peek at the first few rows and the data types
df_preview = pd.read_csv(new_file_path, nrows=5)

print("--- New Dataset Structure ---")
print(df_preview.info())
print("\n--- First 5 Rows ---")
print(df_preview)

--- New Dataset Structure ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   delivery_id    5 non-null      float64
 1   timestamp      5 non-null      object 
 2   revenue        5 non-null      float64
 3   cost           5 non-null      float64
 4   profit         5 non-null      float64
 5   delivery_type  5 non-null      object 
dtypes: float64(4), object(2)
memory usage: 372.0+ bytes
None

--- First 5 Rows ---
   delivery_id            timestamp    revenue       cost     profit  \
0          1.0  2020-06-06 03:47:00  42.240739  10.322332  16.816082   
1          2.0  2024-05-26 11:30:00  15.869323  18.341229  15.294924   
2          3.0  2021-07-18 19:24:00   5.972007   7.685768   7.629064   
3          4.0  2021-10-15 11:56:00  42.108121  19.720245  29.812982   
4          5.0  2020-09-14 19:10:00  15.630525   4.195106   4.388067   

 

In [25]:
import pandas as pd
import numpy as np
import os

input_path = r"C:\Users\noahi\Downloads\delivery_financials.csv\delivery_financials.csv"
output_path = r'C:\Users\noahi\Downloads\delivery_financial_data_PROFESSIONAL.csv'

# 1. Fresh Start
if os.path.exists(output_path):
    os.remove(output_path)

seen_ids = set()
is_first_chunk = True

for chunk in pd.read_csv(input_path, chunksize=100000):
    # Remove blanks and duplicate IDs
    chunk = chunk.dropna(how='all')
    chunk = chunk.drop_duplicates(subset=['delivery_id'])
    chunk = chunk[~chunk['delivery_id'].isin(seen_ids)]
    seen_ids.update(chunk['delivery_id'].tolist())

    # Fix Numeric Columns
    for col in ['revenue', 'cost', 'profit']:
        chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        
        # Use IQR or "Sanity Limits" to remove placeholders (like 999 million)
        # Assuming most deliveries are between $0 and $1000
        median_val = chunk[col].median()
        chunk.loc[chunk[col].abs() > 10000, col] = median_val 
        chunk.loc[np.isinf(chunk[col]), col] = median_val
        chunk[col] = chunk[col].fillna(median_val)

    # THE CRITICAL STEP: Fix the math
    # Professional datasets must have internal consistency
    chunk['profit'] = chunk['revenue'] - chunk['cost']

    # Fix Dates
    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], errors='coerce')

    # Save
    chunk.to_csv(output_path, mode='a', index=False, header=is_first_chunk)
    is_first_chunk = False

print("✅ Financial Dataset Cleaned and Re-Calculated!")

✅ Financial Dataset Cleaned and Re-Calculated!


In [26]:
import pandas as pd

final_finance = r"C:\Users\noahi\Downloads\delivery_financial_data_PROFESSIONAL.csv"
df = pd.read_csv(final_finance)

print("--- FINANCIAL INTEGRITY AUDIT ---")
# This checks if our manual calculation is correct
check = (df['revenue'] - df['cost']) == df['profit']
print(f"Math is 100% correct: {check.all()}")

print("\n--- Profit Overview ---")
print(df['profit'].describe())

--- FINANCIAL INTEGRITY AUDIT ---
Math is 100% correct: False

--- Profit Overview ---
count    833573.000000
mean         17.039317
std          15.270491
min       -1963.945001
25%           5.802179
50%          17.043727
75%          28.274527
max        4981.835907
Name: profit, dtype: float64


In [28]:
import pandas as pd

path = r"C:\Users\noahi\Downloads\delivery_financial_data_PROFESSIONAL.csv"
df = pd.read_csv(path)

# 1. Force the Math & Round to 2 decimals (Cents)
df['revenue'] = df['revenue'].round(2)
df['cost'] = df['cost'].round(2)
df['profit'] = (df['revenue'] - df['cost']).round(2)

# 2. Final Outlier Check (Removing the $4,000 "Glitch" deliveries)
# Realistically, a delivery shouldn't exceed $500 in revenue or cost
for col in ['revenue', 'cost']:
    median_val = df[col].median()
    df.loc[df[col].abs() > 500, col] = median_val

# 3. Recalculate Profit one last time after clipping
df['profit'] = (df['revenue'] - df['cost']).round(2)

# 4. Save and Verify
df.to_csv(path, index=False)

print("--- THE FINAL FINANCIAL AUDIT ---")
check = (df['revenue'] - df['cost']).round(2) == df['profit']
print(f"Math is 100% correct: {check.all()}")
print("\n--- Corrected Profit Stats ---")
print(df['profit'].describe())

--- THE FINAL FINANCIAL AUDIT ---
Math is 100% correct: True

--- Corrected Profit Stats ---
count    833573.000000
mean         17.035732
std          14.103533
min        -191.360000
25%           5.800000
50%          17.040000
75%          28.270000
max         492.500000
Name: profit, dtype: float64


In [29]:
import os

orig_path = r"C:\Users\noahi\Downloads\delivery_financials.csv\delivery_financials.csv"
final_path = r"C:\Users\noahi\Downloads\delivery_financial_data_PROFESSIONAL.csv"

def get_mb(path):
    if os.path.exists(path):
        return f"{os.path.getsize(path) / (1024 * 1024):.2f} MB"
    return "File Not Found"

print(f"{'Dataset Version':<30} | {'File Size':<15}")
print("-" * 50)
print(f"{'Original Financial Data':<30} | {get_mb(orig_path):<15}")
print(f"{'Professional Financial Final':<30} | {get_mb(final_path):<15}")

Dataset Version                | File Size      
--------------------------------------------------
Original Financial Data        | 71.78 MB       
Professional Financial Final   | 41.93 MB       
